# Generalized Huber Regression
This notebook illustrates how to use LightGBM to optimize the Generalized Huber loss (GHL) function with a log link function. Please ensure that you have LightGBM and scikit-learn installed.  

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score
from scipy.stats import skewnorm

As an example we consider a one dimensional linear problem of N data points with a skew normal error distribution and non-constant variance.  

In [2]:
N = 1000000
sigma0 = 1
xvec = np.linspace(-500, 500, N)
yvec = np.array([
    x + skewnorm.rvs(a=100, loc=0, scale=(sigma0 + np.abs(x)**(2)))
    for x in xvec
])

Next we define the log link function and it's inverse.

In [3]:
def log_link_trans(x):
    bool1 = x < 0
    bool2 = x >= 0
    x_trans = np.zeros([1, len(x)]).flatten()
    x_trans[bool1] = -np.log1p(-x[bool1])
    x_trans[bool2] = np.log1p(x[bool2])
    return x_trans


def log_link_back_trans(x):
    bool1 = x < 0
    bool2 = x >= 0
    x_back_trans = np.zeros([1, len(x)]).flatten()
    x_back_trans[bool1] = (1 - np.exp(-x[bool1]))
    x_back_trans[bool2] = (np.exp(x[bool2]) - 1)
    return x_back_trans

We define 2 metric functions which both compute the $r^{2}$ score.

In [4]:
def metric_func_1(yhat, dtrain):
    y = dtrain.get_label().flatten()
    yhat = yhat.flatten()
    yhat_back_trans = log_link_back_trans(yhat)
    y_back_trans = log_link_back_trans(y)
    val = r2_score(y_back_trans, yhat_back_trans)
    return ('r2_score', val, True)


def metric_func_2(yhat, dtrain):
    y = dtrain.get_label()
    yhat = yhat.flatten()
    yhat_back_trans = log_link_back_trans(yhat)
    val = r2_score(y, yhat_back_trans)
    return ('r2_score', val, True)

The next cell defines the objective function for LightGBM by computing the 1st and 2nd derivative of the GHL function.

In [5]:
def generalized_huber_obj(yhat, dtrain, alpha=0.75):
    y = dtrain.get_label()
    yhat = yhat.flatten()

    def sgn(x):
        sig = np.sign(x)
        sig[sig == 0] = 1
        return sig

    g = lambda x: sgn(x) * np.log(1 + np.abs(x))
    ginv = lambda x: sgn(x) * (np.exp(np.abs(x)) - 1)
    ginvp = lambda x: np.exp(np.abs(x))
    ginvpp = lambda x: sgn(x) * np.exp(np.abs(x))

    diff = g(y) - yhat
    absdiff = np.abs(diff)

    bool1_l = ((absdiff <= alpha) & (diff < 0))
    bool1_r = ((absdiff <= alpha) & (diff >= 0))
    bool2_l = ((absdiff > alpha) & (diff < 0))
    bool2_r = ((absdiff > alpha) & (diff >= 0))

    grad = np.zeros([1, len(yhat)]).flatten()
    hess = np.zeros([1, len(yhat)]).flatten()

    A = np.zeros([1, len(yhat)]).flatten()
    Ap = np.zeros([1, len(yhat)]).flatten()
    App = np.zeros([1, len(yhat)]).flatten()
    B = np.zeros([1, len(yhat)]).flatten()

    A[bool1_l] = ginv(yhat[bool1_l] - alpha) - ginv(yhat[bool1_l])
    A[bool1_r] = ginv(yhat[bool1_r] + alpha) - ginv(yhat[bool1_r])
    Ap[bool1_l] = ginvp(yhat[bool1_l] - alpha) - ginvp(yhat[bool1_l])
    Ap[bool1_r] = ginvp(yhat[bool1_r] + alpha) - ginvp(yhat[bool1_r])
    App[bool1_l] = ginvpp(yhat[bool1_l] - alpha) - ginvpp(yhat[bool1_l])
    App[bool1_r] = ginvpp(yhat[bool1_r] + alpha) - ginvpp(yhat[bool1_r])

    A[bool2_l] = ginv(yhat[bool2_l] - alpha) - ginv(yhat[bool2_l])
    A[bool2_r] = ginv(yhat[bool2_r] + alpha) - ginv(yhat[bool2_r])
    Ap[bool2_l] = ginvp(yhat[bool2_l] - alpha) - ginvp(yhat[bool2_l])
    Ap[bool2_r] = ginvp(yhat[bool2_r] + alpha) - ginvp(yhat[bool2_r])
    App[bool2_l] = ginvpp(yhat[bool2_l] - alpha) - ginvpp(yhat[bool2_l])
    App[bool2_r] = ginvpp(yhat[bool2_r] + alpha) - ginvpp(yhat[bool2_r])

    B[bool1_l] = y[bool1_l] - ginv(g(y[bool1_l]) + alpha)
    B[bool1_r] = y[bool1_r] - ginv(g(y[bool1_r]) - alpha)

    grad[bool1_l] = -2*(y[bool1_l]-ginv(yhat[bool1_l]))*ginvp(yhat[bool1_l])*(1/np.abs(A[bool1_l]) + 1/np.abs(B[bool1_l])) \
                    -(y[bool1_l]-ginv(yhat[bool1_l]))**2*(1/(np.abs(A[bool1_l])**2))*sgn(A[bool1_l])*Ap[bool1_l]

    grad[bool1_r] = -2*(y[bool1_r]-ginv(yhat[bool1_r]))*ginvp(yhat[bool1_r])*(1/np.abs(A[bool1_r]) + 1/np.abs(B[bool1_r])) \
                    -(y[bool1_r]-ginv(yhat[bool1_r]))**2*(1/(np.abs(A[bool1_r])**2))*sgn(A[bool1_r])*Ap[bool1_r]

    hess[bool1_l] = 2*(ginvp(yhat[bool1_l])**2 - (y[bool1_l]-ginv(yhat[bool1_l]))*ginvpp(yhat[bool1_l]))*(1/np.abs(A[bool1_l]) + 1/np.abs(B[bool1_l])) \
                    +4*(y[bool1_l]-ginv(yhat[bool1_l]))*ginvp(yhat[bool1_l])*(1/(np.abs(A[bool1_l])**2))*sgn(A[bool1_l])*Ap[bool1_l] \
                    +2*(y[bool1_l]-ginv(yhat[bool1_l]))**2*(1/(np.abs(A[bool1_l])**3))*Ap[bool1_l]**2 \
                    -(y[bool1_l]-ginv(yhat[bool1_l]))**2*(1/(np.abs(A[bool1_l])**2))*sgn(A[bool1_l])*App[bool1_l]

    hess[bool1_r] = 2*(ginvp(yhat[bool1_r])**2 - (y[bool1_r]-ginv(yhat[bool1_r]))*ginvpp(yhat[bool1_r]))*(1/np.abs(A[bool1_r]) + 1/np.abs(B[bool1_r])) \
                   +4*(y[bool1_r]-ginv(yhat[bool1_r]))*ginvp(yhat[bool1_r])*(1/(np.abs(A[bool1_r])**2))*sgn(A[bool1_r])*Ap[bool1_r] \
                   +2*(y[bool1_r]-ginv(yhat[bool1_r]))**2*(1/(np.abs(A[bool1_r])**3))*Ap[bool1_r]**2 \
                    -(y[bool1_r]-ginv(yhat[bool1_r]))**2*(1/(np.abs(A[bool1_r])**2))*sgn(A[bool1_r])*App[bool1_r]

    grad[bool2_l] = -4 * sgn(y[bool2_l] - ginv(yhat[bool2_l])) * ginvp(
        yhat[bool2_l]) - sgn(A[bool2_l]) * Ap[bool2_l]

    grad[bool2_r] = -4 * sgn(y[bool2_r] - ginv(yhat[bool2_r])) * ginvp(
        yhat[bool2_r]) - sgn(A[bool2_r]) * Ap[bool2_r]

    hess[bool2_l] = -4 * sgn(y[bool2_l] - ginv(yhat[bool2_l])) * ginvpp(
        yhat[bool2_l]) - sgn(A[bool2_l]) * App[bool2_l]

    hess[bool2_r] = -4 * sgn(y[bool2_r] - ginv(yhat[bool2_r])) * ginvpp(
        yhat[bool2_r]) - sgn(A[bool2_r]) * App[bool2_r]

    return grad, hess

Next we put our data into a Dataframe, define the train and test sets and compute 3 cv folds.

In [6]:
df = pd.DataFrame(np.column_stack((xvec, yvec)), columns=['x', 'y'])

X_train, X_test = train_test_split(df, test_size=0.3)
y_train = X_train['y']
y_test = X_test['y']

groups = X_train.index
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(X_train, groups)
folds = list(group_kfold.split(X_train, y_train, groups=groups))

We start by training a shallow Lightgbm model whose output will be the starting vector of the GHL model.

In [7]:
lgb_param_start_vector = {
    'learning_rate': 0.1,
    'num_leaves': 2**5 - 1,
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'feature_fraction': 1,
    'alpha': 2,
    'objective': 'huber',
    'metric': 'none',
    'tree_learner': 'data',
    'verbosity': 3
}

lgbtrain_trans = lgb.Dataset(
    X_train.drop('y', axis=1), label=log_link_trans(y_train))
lgbmodel0 = lgb.train(
    lgb_param_start_vector, lgbtrain_trans, num_boost_round=20)

start_vec = lgbmodel0.predict(X_train.drop('y', axis=1))
lgbtrain = lgb.Dataset(
    X_train.drop('y', axis=1), init_score=start_vec, label=y_train)

Next we perform a hyperparameter search to find the optimal $\alpha$ parameter in the GHL function.

In [8]:
lgb_param_ghl = {
    'learning_rate': 0.1,
    'num_leaves': 2**5 - 1,
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'feature_fraction': 1,
    'metric': 'none',
    'max_delta_step': 3,
    'verbosity': 1
}

kwargsvec = np.linspace(0.1, 1, 20)
chvec = np.zeros(20)
for k in range(20):
    kwargs = {'alpha': kwargsvec[k]}
    cvmodel = lgb.cv(
        lgb_param_ghl,
        lgbtrain,
        num_boost_round=10000,
        folds=folds,
        fobj=(lambda a, b: generalized_huber_obj(a, b, **kwargs)),
        metrics='none',
        feval=metric_func_2,
        early_stopping_rounds=20,
        verbose_eval=False)
    chvec[k] = cvmodel['r2_score-mean'][-1]
    print(kwargsvec[k])

best_iter = np.argmax(chvec)
alpha_ch = kwargsvec[best_iter]

0.1
0.147368421053
0.194736842105
0.242105263158
0.289473684211
0.336842105263
0.384210526316
0.431578947368
0.478947368421
0.526315789474
0.573684210526
0.621052631579
0.668421052632
0.715789473684
0.763157894737
0.810526315789
0.857894736842
0.905263157895
0.952631578947
1.0


Having found the optimal $\alpha$ we can now train the GHL model.

In [9]:
kwargs = {'alpha': alpha_ch}
cvmodel = lgb.cv(
    lgb_param_ghl,
    lgbtrain,
    num_boost_round=10000,
    folds=folds,
    fobj=(lambda a, b: generalized_huber_obj(a, b, **kwargs)),
    metrics='none',
    feval=metric_func_2,
    early_stopping_rounds=20,
    verbose_eval=True)

print('generalized huber best alpha: ', alpha_ch)
print('generalized huber best iteration: ', len(cvmodel['r2_score-mean']))

lgbmodel = lgb.train(
    lgb_param_ghl,
    lgbtrain,
    num_boost_round=len(cvmodel['r2_score-mean']),
    fobj=(lambda a, b: generalized_huber_obj(a, b, **kwargs)),
    feval=metric_func_2,
    verbose_eval=True)

[1]	cv_agg's r2_score: 0.315028 + 0.000139703
[2]	cv_agg's r2_score: 0.342772 + 0.000454857
[3]	cv_agg's r2_score: 0.361445 + 0.000666957
[4]	cv_agg's r2_score: 0.37504 + 0.000807633
[5]	cv_agg's r2_score: 0.385405 + 0.000917626
[6]	cv_agg's r2_score: 0.393495 + 0.000991963
[7]	cv_agg's r2_score: 0.399925 + 0.00104371
[8]	cv_agg's r2_score: 0.40511 + 0.00108262
[9]	cv_agg's r2_score: 0.409341 + 0.00111595
[10]	cv_agg's r2_score: 0.412822 + 0.00114632
[11]	cv_agg's r2_score: 0.41571 + 0.00116981
[12]	cv_agg's r2_score: 0.418119 + 0.00119227
[13]	cv_agg's r2_score: 0.420144 + 0.00120966
[14]	cv_agg's r2_score: 0.421857 + 0.00122312
[15]	cv_agg's r2_score: 0.42331 + 0.00123593
[16]	cv_agg's r2_score: 0.424551 + 0.00124597
[17]	cv_agg's r2_score: 0.425614 + 0.0012578
[18]	cv_agg's r2_score: 0.426531 + 0.00126598
[19]	cv_agg's r2_score: 0.427323 + 0.00127319
[20]	cv_agg's r2_score: 0.428007 + 0.00127968
[21]	cv_agg's r2_score: 0.428602 + 0.00128593
[22]	cv_agg's r2_score: 0.429119 + 0.00129

Next a MAE model is trained on $\log(y)$ ...

In [10]:
lgb_param_mae = {
    'learning_rate': 0.1,
    'num_leaves': 2**5 - 1,
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'feature_fraction': 1,
    'objective': 'mae',
    'metric': 'none',
    'tree_learner': 'data',
    'verbosity': 1
}

cvmodel_mae = lgb.cv(
    lgb_param_mae,
    lgbtrain_trans,
    num_boost_round=10000,
    folds=folds,
    feval=metric_func_1,
    metrics='none',
    early_stopping_rounds=20,
    verbose_eval=True)

lgbmodel_mae = lgb.train(
    lgb_param_mae,
    lgbtrain_trans,
    num_boost_round=len(cvmodel_mae['r2_score-mean']),
    verbose_eval=True)

[1]	cv_agg's r2_score: -0.110404 + 0.000665364
[2]	cv_agg's r2_score: -0.0615394 + 0.000558223
[3]	cv_agg's r2_score: -0.0163966 + 0.000508024
[4]	cv_agg's r2_score: 0.0256555 + 0.000535871
[5]	cv_agg's r2_score: 0.0648368 + 0.000601036
[6]	cv_agg's r2_score: 0.101102 + 0.000615428
[7]	cv_agg's r2_score: 0.134598 + 0.000661443
[8]	cv_agg's r2_score: 0.165353 + 0.00068997
[9]	cv_agg's r2_score: 0.193312 + 0.000683287
[10]	cv_agg's r2_score: 0.21857 + 0.000695577
[11]	cv_agg's r2_score: 0.24124 + 0.000712882
[12]	cv_agg's r2_score: 0.261572 + 0.000753481
[13]	cv_agg's r2_score: 0.279689 + 0.000762501
[14]	cv_agg's r2_score: 0.295759 + 0.000771255
[15]	cv_agg's r2_score: 0.309956 + 0.000768499
[16]	cv_agg's r2_score: 0.322511 + 0.000808143
[17]	cv_agg's r2_score: 0.333585 + 0.000824778
[18]	cv_agg's r2_score: 0.343312 + 0.000814279
[19]	cv_agg's r2_score: 0.351867 + 0.000838262
[20]	cv_agg's r2_score: 0.359355 + 0.00083988
[21]	cv_agg's r2_score: 0.365936 + 0.000838567
[22]	cv_agg's r2_sc

... as well as a RMSE model. 

In [11]:
lgb_param_rmse = {
    'learning_rate': 0.1,
    'num_leaves': 2**5 - 1,
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'feature_fraction': 1,
    'objective': 'rmse',
    'metric': 'none',
    'tree_learner': 'data',
    'verbosity': 1
}

cvmodel_rmse = lgb.cv(
    lgb_param_rmse,
    lgbtrain_trans,
    num_boost_round=10000,
    folds=folds,
    feval=metric_func_1,
    metrics='none',
    early_stopping_rounds=20,
    verbose_eval=True)

lgbmodel_rmse = lgb.train(
    lgb_param_rmse,
    lgbtrain_trans,
    num_boost_round=len(cvmodel_rmse['r2_score-mean']),
    verbose_eval=True)

[1]	cv_agg's r2_score: -0.260506 + 0.00121804
[2]	cv_agg's r2_score: -0.223009 + 0.00111538
[3]	cv_agg's r2_score: -0.186053 + 0.001013
[4]	cv_agg's r2_score: -0.149712 + 0.000919921
[5]	cv_agg's r2_score: -0.114196 + 0.000803888
[6]	cv_agg's r2_score: -0.0799775 + 0.000763266
[7]	cv_agg's r2_score: -0.0471802 + 0.000649524
[8]	cv_agg's r2_score: -0.016189 + 0.000623505
[9]	cv_agg's r2_score: 0.012932 + 0.000539302
[10]	cv_agg's r2_score: 0.0401511 + 0.000450631
[11]	cv_agg's r2_score: 0.0654257 + 0.00040275
[12]	cv_agg's r2_score: 0.0886681 + 0.000453112
[13]	cv_agg's r2_score: 0.11001 + 0.000484833
[14]	cv_agg's r2_score: 0.129551 + 0.000534348
[15]	cv_agg's r2_score: 0.147344 + 0.000554739
[16]	cv_agg's r2_score: 0.163457 + 0.000615147
[17]	cv_agg's r2_score: 0.178106 + 0.000629504
[18]	cv_agg's r2_score: 0.191377 + 0.000658537
[19]	cv_agg's r2_score: 0.203336 + 0.000685421
[20]	cv_agg's r2_score: 0.214115 + 0.000731218
[21]	cv_agg's r2_score: 0.223823 + 0.000774406
[22]	cv_agg's r2

[178]	cv_agg's r2_score: 0.309755 + 0.00108985
[179]	cv_agg's r2_score: 0.309755 + 0.00108995
[180]	cv_agg's r2_score: 0.309755 + 0.00108996
[181]	cv_agg's r2_score: 0.309755 + 0.00108991


We can now compute the predictions of all 3 models on the train and test set.

In [12]:
start_vec_test = lgbmodel0.predict(X_test.drop('y', axis=1))
scores_ghl_test = lgbmodel.predict(X_test.drop('y', axis=1))
scores_ghl_test = log_link_back_trans(scores_ghl_test + start_vec_test)

scores_ghl_train = lgbmodel.predict(X_train.drop('y', axis=1))
scores_ghl_train = log_link_back_trans(scores_ghl_train + start_vec)

scores_rmse_train = lgbmodel_rmse.predict(X_train.drop('y', axis=1))
scores_rmse_train = log_link_back_trans(scores_rmse_train)

scores_rmse_test = lgbmodel_rmse.predict(X_test.drop('y', axis=1))
scores_rmse_test = log_link_back_trans(scores_rmse_test)

scores_mae_train = lgbmodel_mae.predict(X_train.drop('y', axis=1))
scores_mae_train = log_link_back_trans(scores_mae_train)

scores_mae_test = lgbmodel_mae.predict(X_test.drop('y', axis=1))
scores_mae_test = log_link_back_trans(scores_mae_test)

With the predictions at hand we finally compute the $r^{2}$ score and the realtive error of the global mean.

In [13]:
print("mean error train ghl = ",
      (y_train.mean() - scores_ghl_train.mean()) / (y_train.mean()))
print("mean error test ghl  = ",
      (y_test.mean() - scores_ghl_test.mean()) / (y_test.mean()))
print("mean error train mae = ",
      (y_train.mean() - scores_mae_train.mean()) / (y_train.mean()))
print("mean error test mae = ",
      (y_test.mean() - scores_mae_test.mean()) / (y_test.mean()))
print("mean error train rmse = ",
      (y_train.mean() - scores_rmse_train.mean()) / (y_train.mean()))
print("mean error test rmse = ",
      (y_test.mean() - scores_rmse_test.mean()) / (y_test.mean()))
print("             ..........          ")
print("r2 score train ghl = ", r2_score(y_train, scores_ghl_train))
print("r2 scores test ghl  = ", r2_score(y_test, scores_ghl_test))
print("r2 score train mae = ", r2_score(y_train, scores_mae_train))
print("r2 score test mae = ", r2_score(y_test, scores_mae_test))
print("r2 score train rmse = ", r2_score(y_train, scores_rmse_train))
print("r2 score test rmse = ", r2_score(y_test, scores_rmse_test))

mean error train ghl =  0.0769479300871
mean error test ghl  =  0.0750770768387
mean error train mae =  0.153917810958
mean error test mae =  0.152216327981
mean error train rmse =  0.361297656997
mean error test rmse =  0.359984995732
             ..........          
r2 score train ghl =  0.433539836501
r2 scores test ghl  =  0.430126436051
r2 score train mae =  0.416110276056
r2 score test mae =  0.413121524319
r2 score train rmse =  0.310169634178
r2 score test rmse =  0.307997141499
